## Setup

To run these examples, you'll need an IonQ API key.

As before, we'll make sure the API key is set up as an environment variable:

In [ ]:
import os
os.environ["IONQ_API_KEY"] = "YOUR API KEY HERE"

We'll use some built-in Python libraries as well (though you can work with the API directly in the command line or using other programming languages).

In [ ]:
import json
import requests

## API v0.4

We're in the process of launching API v0.4 and upgrading our platform and integrations from API v0.3. We'll continue to support v0.3, and much of v0.4 will look similar, but there are some differences.

Using the API is not required (you may find that an SDK is more convenient, and is sufficient for everything you need to do), but knowing what it does can provide a deeper understanding of how our SDK integrations and our systems work, and can provide access to some capabilities that aren't fully integrated into all SDKs.

## Check access and credentials

To check if we can connect to the API, we'll send a `GET` request to an endpoint that doesn't require authorization. If the response code is 200, this worked.

In [ ]:
response = requests.get("https://api.ionq.co/v0.4/backends")

In [ ]:
response

You might also see examples with slightly different formatting, as in our docs. Here, `requests.get(...)` is equivalent to `requests.request("GET"...)`.

In [ ]:
response = requests.request("GET", "https://api.ionq.co/v0.4/backends")

In [ ]:
response

Next, set up the API request headers, which will include our authentication (API key). Here we're retrieving the key that was stored as an environment variable above (or external to this notebook), but you could also just put your API key here. We'll use the same headers for all requests that require authentication.

In [ ]:
headers = {
    "Authorization": f"apiKey {os.getenv('IONQ_API_KEY')}",
    "Content-Type": "application/json",
}

Look at the headers (note that this prints your API key):

In [ ]:
headers

## API key info

The "who am I" API endpoint returns information about the API key that was used to send the request. This can be especially useful if you're managing multiple API keys from different projects, and you want to confirm that jobs submitted with this API key will be sent to the correct project.

In [ ]:
response_whoami = requests.get(
    f"https://api.ionq.co/v0.4/whoami",
    headers=headers
)

View the response as json (effectively a Python dictionary):

In [ ]:
response_whoami.json()

This gives us a unique identifier for the key, the name we gave the key when we created it, and the unique identifier for the project it's tied to.

Check the cloud console to see which project this key corresponds to.

In [ ]:
print(f"https://cloud.ionq.com/projects/{response_whoami.json()['project_id']}")

In [ ]:
job_data = {
    "name": "API example - sim",
    "type": "ionq.circuit.v1",
    
    "backend": "simulator",
    "noise": {"model": "ideal"},
    
    "input": {
        "qubits": 2,
        "gateset": "qis",
        "circuit": [
            {"gate": "h", "target": 0},
            {"gate": "x", "target": 1, "control": 0}
        ]
    },
}

Send the request:

In [ ]:
response_create_job = requests.post(
    "https://api.ionq.co/v0.4/jobs",
    headers=headers,
    json=job_data
)

The response (if the job was submitted successfully) has the job ID. A response code of 404 usually means you don't have the right credentials, access, permissions, etc. while a code of 400 usually means an issue with the syntax or structure. 201 means the job was submitted successfully, though it can still fail during subsequent steps.

In [ ]:
response_create_job

Look at the response body to get the status and job ID:

In [ ]:
response_create_job.json()

The job ID is populated as soon as the job is submitted, but we have to send another request to get the updated status for this job. Save the job ID, from the create job response or the cloud console:

In [ ]:
job_id_sim = response_create_job.json()['id']
print(job_id_sim)

This is a `get` request to the endpoint `jobs/MY_JOB_ID`. We'll put the job ID into the URL and include the headers with the API key, but there is no data or other payload in this request.

In [ ]:
response_get_job = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_sim}",
    headers=headers
)

First look at the status:

In [ ]:
response_get_job.json()['status']

The full response contains a lot of other information about the job - some fields are based on your submission, some were populated by IonQ's cloud platform. Some job-related information, like the job cost and results, uses different API endpoints (see below).

In [ ]:
response_get_job.json()

The result probabilities can be requested using a specific URL. This is included in the response above, but you can also plug the job ID into the URL structure.

In [ ]:
job_result_url = "https://api.ionq.co" + response_get_job.json()['results']['probabilities']['url']

In [ ]:
job_result_url = f"https://api.ionq.co/v0.4/jobs/{job_id_sim}/results/probabilities"

In [ ]:
print(job_result_url)

Send a GET request to this URL:

In [ ]:
response_get_result = requests.get(
    job_result_url,
    headers=headers
)

This response is just a dictionary containing the probabilities for each state:

In [ ]:
response_get_result.json()

In the future, `results` may include additional fields depending on the job type and result format.

In [ ]:
job_data_qpu = {
    "name": "API example - Aria 1",
    "type": "ionq.circuit.v1",

    "backend": "qpu.aria-1",
    "shots": 100,
    
    "input": {
        "qubits": 2,
        "gateset": "qis",
        "circuit": [
            {"gate": "h", "target": 0},
            {"gate": "x", "target": 1, "control": 0}
        ]
    },
}

In [ ]:
response_create_job_qpu = requests.post(
    "https://api.ionq.co/v0.4/jobs",
    headers=headers,
    json=job_data_qpu
)

In [ ]:
response_create_job_qpu.json()

In [ ]:
job_id_qpu1 = response_create_job_qpu.json()['id']

As before, let's set up a request to get the job status. We'll confirm that the job went from "submitted" to "ready" (queued).

In [ ]:
response_get_job_qpu = requests.get(
    f"https://api.ionq.co/v0.4/jobs/{job_id_qpu1}",
    headers=headers
)

In [ ]:
response_get_job_qpu.json()['status']

The full response for this job includes a lot of information, similar to the simulator job example above - but it doesn't include the result URL, because the result doesn't exist yet. Instead it shows that the result is `None`, for now.

Submit the job:

# Backends

The API also includes several endpoints related to backend (simulator and QPU) status and characterization information. Some of these are also available without an API key.

### Get current info for all backends

Sending a GET request to the /backends endpoint returns a list of backends, with some status and queue information.

In [ ]:
response_get_backends = requests.get(
    "https://api.ionq.co/v0.4/backends",
    headers=headers
)

In [ ]:
response_get_backends.json()

### Get a specific backend

Similarly, /backends/SPECIFIC_QPU_NAME gives this information only for the specified backend. Let's look at Aria 1:

In [ ]:
response_get_aria1 = requests.get(
    f"https://api.ionq.co/v0.4/backends/qpu.aria-1",
    headers=headers
)

In [ ]:
response_get_aria1.json()

Average queue time is in milliseconds. Since we're using a reservation, our jobs will skip the queue during this workshop, and we won't have to wait this long to run. In general, fair-share queueing means that your queue time might also be much shorter - for example, if the queue time was long because one or two users were running very large workloads, jobs from other users would likely get to skip at least part of the line.

In [ ]:
response_get_aria1.json()['average_queue_time']/1000/60

Last updated time is in UTC.

### Get characterization data

The backend info includes a unique identifier for the system characterization, which we can use to pull recently recorded characterization data for the system.

To get a characterization from its ID, use `/backends/QPU_NAME/characterizations/CHARACTERIZATION_ID`.

In [ ]:
char_id = response_get_aria1.json()['characterization_id']

In [ ]:
response_get_char = requests.get(
    f"https://api.ionq.co/v0.4/backends/qpu.aria-1/characterizations/{char_id}",
    headers=headers
)

This response includes the date and time this dataset was updated (typically midnight UTC), as well as other information about the system.

In [ ]:
response_get_char.json()

The most important (and most frequently updated) information is the fidelity, which shows the median error for 1Q DRB, 2Q DRB, and SPAM. These values represent a median of the data points recorded for different qubits and qubit pairs over a 24-hour period.

In [ ]:
response_get_char.json()['fidelity']

In [ ]:
response_get_char.json()['date']

### Get multiple characterizations

We can also filter for characterization data from a specific date range. For example, if we want to look at all characterization data for Aria 1 between July 1 and July 10, we can specify start and end dates.

In [ ]:
response_get_chars = requests.get(
    f"https://api.ionq.co/v0.4/backends/qpu.aria-1/characterizations",
    headers=headers,
    params={'start': '2025-07-01', 'end': '2025-07-10'}
)

We can loop over the data and pull out the date and fidelity for each stored characterization:

In [ ]:
for ch in response_get_chars.json()['characterizations']:
    print(ch['date'])
    print(ch['fidelity'])
    print()

## Wrap-up

While API v0.4 is currently in beta and not all endpoints are fully documented, it will be officially released soon. More job types, result formats, and settings will also become available in the future. For now, we'd appreciate your feedback on 

https://docs.ionq.com/api-reference/v0.4/introduction